<a href="https://colab.research.google.com/github/lena110200/Class2022-spring/blob/master/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

### [Image classification](https://huggingface.co/tasks/image-classification)

In [ ]:
# Image classification is the task of assigning a label or class to an entire image. 
#Images are expected to have only one class for each image.Image classification 
#models take an image as input and return a prediction about which class the image belongs to.

e.g. https://huggingface.co/google/vit-base-patch16-224 \
:model
How to use

#웹페이지에 image classification정의 나옴 

In [ ]:
!pip install transformers # transformer 먼저 install시켜줘야 

In [3]:
#how to use copy / image를 classify함 . 어떤 class가 확률이 높은지 맞춰줌 
#대충 읽을 수 는 있어야 
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests  # 필요한거 설치하는 부분 

url = 'http://images.cocodataset.org/val2017/000000039769.jpg' #url안에 그림/ 이그림을 인식해라 
image = Image.open(requests.get(url, stream=True).raw) #이미지를 open해서. image안에 숫자형태로 저장
#jpg에 있는 정보가 숫자 형태로 들어옴 
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
#pretrained : 데이터로부터 훈련된 행렬들 - feature extraction을 할수있는 모델 
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')#extract된 feature정보를 classify

inputs = feature_extractor(images=image, return_tensors="pt") #이미지 정보로부터 feature정보를 뽑아냄
#original image보다 input이 숫자가 적음. 
outputs = model(**inputs) #모델이 image input을 가지고 와서 분석 / output이 classify된게 나옴
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes - 총 천개의 클래스에 대해 확률정보 계산(1000개의 숫자가 나오는것)
#- 총천개의 클래스 중에 하나를 predict 
#천개 숫자 다더하면 1 
predicted_class_idx = logits.argmax(-1).item() #천개의 숫자중 확률이 젤 큰거 찾아라 - 한개 
print("Predicted class:", model.config.id2label[predicted_class_idx])

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


Demo in Gradio

In [4]:
def func (image): #위에 있는 script를 가지고 일부를 떼서 function화 
#이미지를 받아서 class 를 text로 return하는 함수 
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx] #출력 . 계산된 class 
  return predicted_class

In [5]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")
#계정에 다운로드를 해줌 

0

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#gradio : 위의 함수의 입출력의 유형을 정의 해줘야. 이미지를 넣고 출력으로 text. 
#이미지 파일 두개가 있다는걸 assume / 예를 올려두면 편해 / 파일이름을 반드시 list로 해서 적어줌
#- 데모를 보는 사람들이 쉽게 클릭해서 볼 수 있도록  

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

In [ ]:
#Masked language modeling is the task of masking some of the words in a sentence 
# and predicting which words should replace those masks. 
# These models are useful when we want to get a statistical understanding of the language in which the model is trained in.

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [7]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased') #모델을 들고와서 이름을 unmasker로 하자 
unmasker("Hello I'm a [MASK] model.") 
#dict가 list로 묶여서 나옴 / list에 item이 5개 

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

Demo in Gradio

In [8]:
import pandas as pd
def func (text): #'Hello I'm a [MASK] model' 이 text를 입력으로 받아서 
  unmasker = pipeline('fill-mask', model='bert-base-uncased') #모델 불러와서 
  result = unmasker(text) #
  df = pd.DataFrame(result) # 위의 list를 dataframe화 .엑셀형식으로 
  return df

In [9]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()
#위의 잇는 예문들을 list에 넣어  

### [Token classification](https://huggingface.co/tasks/token-classification)

In [ ]:
#natural lg understanding - natural lg가 주어져있다면 그게 자유도가 높은데(막이야기) 필요한 정보만 뽑아내야.
#NER (named entity recognition) : 문장이 주어져있을때 어떤 명사는 더 정보적으로 중요. 더 관심을가져야. 그런것들을 인식. 
# 위치 정보(berlin), 단체정보, 사람, misc.. 

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")#str형태의 문장을 단어수준으로 쪼개주는
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#각 token 단어들을 classify - NER(4가지종류 인식)
nlp = pipeline("ner", model=model, tokenizer=tokenizer) #nlp가 모델.모델을 한꺼번에 모아주는.
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example) #위의 있는 문장을 가지고 모델을 돌림 
print(ner_results) # 각 토큰에 대해 설명 / dict가 두개가 list로 담겨져있음

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


Demo in Gradio

In [13]:
#gradio만들기
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result) #최종 result를 df로 받아 / list로 dict가 여러개 받아져있을때 바로 df화할수있음
  return df

In [14]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
#Sentence Similarity is the task of determining how similar two texts are. 
#Sentence similarity models convert input texts into vectors (embeddings) 
#that capture semantic information and calculate how close (similar) they are between them. 
#This task is particularly useful for information retrieval and clustering/grouping.

In [ ]:
# 두 주어진 문장이 있을때 얼만큼 유사한가 / 모든 모델들은 input을 바로 받아들일 수 없음 - 중간단계 필요(feature vector embedding)
#raw data를 모델이 그대로 받아들이기 힘듬. feature형태로 . embedding형태로 바꿔줘 
#embedding형태로 바꿔줘야 두 text간의 similarity를 쉽게 계산할수있음

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)#모델을 가지고 입력으로 위의 문장을 써서 embeddings를 뱉어줌
print(embeddings) 
#embiddings : list 2개 / 숫자 총 384 / 384개의 숫자가 [...] [...] : []하나가 embedding 
#list안에 있는게 feature vector 숫자들 data -> 이걸 가지고 similarity계산 

In [18]:
len(embeddings[1]) #길이 2 - list가 2개 들었으니 
#첫번째 list길이는 384

384

In [19]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1]) # 첫번째 문장에 대한 384개의 숫자, 두번째 문장 ~
#cos similarity
#두백타가 이루는 각도의 코사인 계산 
cosine_scores

tensor([[0.5726]])

Demo in Gradio

In [20]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [21]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()